In [2]:
import pandas as pd

In [3]:
schoolfile = "../Resources/schools_complete.csv"
studentfile = "../Resources/students_complete.csv"

schooldata = pd.read_csv(schoolfile)
studentdata = pd.read_csv(studentfile)

In [4]:
df = pd.merge(studentdata, schooldata, how="left", on=["school_name","school_name"])
df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [121]:
# DistrictSummary
totalschools = df['School ID'].nunique()
totalstudents = df['Student ID'].nunique()
totalbudget = schooldata['budget'].sum()
avgmath = df['math_score'].mean()
avgmath = round(df['math_score'].mean(),2)
avgread = round(df['reading_score'].mean(),2)
mathpassperc = round(((studentdata['math_score']>=70).sum()/studentdata['math_score'].count())*100,2)
readpassperc = round(((studentdata['reading_score']>=70).sum()/studentdata['reading_score'].count())*100,2)

mathdf = studentdata.loc[(studentdata["math_score"]>=70)]
overalldf = mathdf.loc[(mathdf["reading_score"]>=70)]
overallpass = round((overalldf["Student ID"].count()/totalstudents)*100,2)

DistrictSummary_df = pd.DataFrame({"Total Schools": totalschools,
                                   "Total Students": totalstudents,
                                   "Total Budget": totalbudget,
                                   "Avg Math Score": avgmath,
                                   "Avg Read Score": avgread,
                                   "% Passing Math": mathpassperc,
                                   "% Passing Reading": readpassperc,
                                   "% Passing Overall": overallpass}, index=[0])
DistrictSummary_df["Total Students"] = DistrictSummary_df["Total Students"].map("{:,}".format)
DistrictSummary_df["Total Budget"] = DistrictSummary_df["Total Budget"].map("${:,}".format)
DistrictSummary_df.head()

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Read Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,65.17


In [224]:
# SchoolSummary
summdf = df.groupby(["school_name"])
schoolnames = df['school_name'].sort_values().unique()
schooltype = schooldata.sort_values(by="school_name").type
students = list(summdf['student_name'].count())
budget = list(summdf['budget'].mean())
perstudentbudg = [i/j for i,j in zip(budget,students)]
avgmath = list(summdf['math_score'].mean())
avgread = list(summdf['reading_score'].mean())

summmath = df[df['math_score']>=70].groupby(['school_name'])
mathpass = [(i/j)*100 for i,j in zip(summmath['math_score'].count(),students)]
summread = df[df['reading_score']>=70].groupby(['school_name'])
readpass = [(i/j)*100 for i,j in zip(summread['reading_score'].count(),students)]

newdf = df[df['math_score']>=70]
newdf = newdf[newdf['reading_score']>=70].groupby(['school_name'])
overallpass = [(i/j)*100 for i,j in zip(newdf['reading_score'].count(),students)]

SchoolSummary_df = pd.DataFrame({"School Names": schoolnames,
                                "School Type": schooltype,
                                "Total Students": students,
                                "Total School Budget": budget,
                                "Per Student Budget": perstudentbudg,
                                "Avg Math Score": avgmath,
                                "Avg Reading Score": avgread,
                                "% Passing Math": mathpass,
                                "% Passing Reading": readpass,
                                "% Overall Passing": overallpass})
SchoolSummary_df["Total Students"] = SchoolSummary_df["Total Students"].astype(int)
# SchoolSummary_df["Total Students"] = SchoolSummary_df["Total Students"].map("{:,}".format)
SchoolSummary_df["Total School Budget"] = SchoolSummary_df["Total School Budget"].map("${:,}".format)
SchoolSummary_df["Per Student Budget"] = SchoolSummary_df["Per Student Budget"].astype(int).map("{:,}".format)
# SchoolSummary_df["Per Student Budget"] = SchoolSummary_df["Per Student Budget"].map("${:,}".format)
SchoolSummary_df = SchoolSummary_df.reset_index(drop=True)
SchoolSummary_df.head()

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.0",628,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.0",582,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411.0",639,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916.0",644,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.0",625,83.351499,83.816757,93.392371,97.138965,90.599455


In [159]:
#Sort the top 5 performing schools by % overall passing
top5schools = SchoolSummary_df.sort_values(by='% Overall Passing', ascending=False)
top5schools = top5schools.reset_index(drop=True)
top5schools.head(5)

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,"1,858","$1,081,356.0",$582.0,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,"1,635","$1,043,130.0",$638.0,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,"1,468","$917,500.0",$625.0,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,"2,283","$1,319,574.0",$578.0,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,"$585,858.0",$609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [160]:
#Sort the bottom 5 performing schools by % overall passing
bottom5schools = SchoolSummary_df.sort_values(by='% Overall Passing', ascending=True)
bottom5schools = bottom5schools.reset_index(drop=True)
bottom5schools.head(5)

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,"3,999","$2,547,363.0",$637.0,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,"2,949","$1,884,411.0",$639.0,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,"2,917","$1,910,635.0",$655.0,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,"4,635","$3,022,020.0",$652.0,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,"4,761","$3,094,650.0",$650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [167]:
#Math scores by grade
gr9_df = df.loc[df['grade'] == "9th"]
gr10_df = df.loc[df['grade'] == "10th"]
gr11_df = df.loc[df['grade'] == "11th"]
gr12_df = df.loc[df['grade'] == "12th"]

gr9math = gr9_df.groupby(['school_name']).mean()['math_score']
gr10math = gr10_df.groupby(['school_name']).mean()['math_score']
gr11math = gr11_df.groupby(['school_name']).mean()['math_score']
gr12math = gr12_df.groupby(['school_name']).mean()['math_score']

mathsumm_df = pd.DataFrame({"9th":gr9math,
                           "10th":gr10math,
                           "11th":gr11math,
                           "12th":gr12math})
mathsumm_df.head(10)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [168]:
#Reading scores by grade
gr9read = gr9_df.groupby(['school_name']).mean()['reading_score']
gr10read = gr10_df.groupby(['school_name']).mean()['reading_score']
gr11read = gr11_df.groupby(['school_name']).mean()['reading_score']
gr12read = gr12_df.groupby(['school_name']).mean()['reading_score']

readsumm_df = pd.DataFrame({"9th":gr9read,
                           "10th":gr10read,
                           "11th":gr11read,
                           "12th":gr12read})
readsumm_df.head(10)

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [218]:
#Scores by school spending
bins = [0,585,630,645,680]
bin_names = ["<$585","$585-629","$630-644","$645-679"]

Spending_summary = SchoolSummary_df[["School Names","Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Spending_summary["Spending Bins"] = pd.cut(pd.to_numeric(SchoolSummary_df["Per Student Budget"]), bins, labels=bin_names)
Spending_summary = Spending_summary.groupby(["Spending Bins"])
Spending_summary_df = Spending_summary.mean()
Spending_summary_df.head()

C:\Users\schow\AppData\Local\Temp\ipykernel_12736\2017071388.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Spending_summary["Spending Bins"] = pd.cut(pd.to_numeric(SchoolSummary_df["Per Student Budget"]), bins, labels=bin_names)


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Bins,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,81.899826,83.155286,87.133538,92.718205,81.418596
$630-644,78.518855,81.624473,73.484209,84.391793,62.857656
$645-679,76.997210,81.027843,66.164813,81.133951,53.526855


In [225]:
#Scores by school size
bins = [0,1000,2000,5000]
bin_names = ["Small (<1000)","Medium (1000-2000)","Large (2000-5000)"]

Size_summary = SchoolSummary_df[["School Names","Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Size_summary["Size Bins"] = pd.cut(SchoolSummary_df["Total Students"], bins, labels=bin_names)
Size_summary = Size_summary.groupby(["Size Bins"])
Size_summary_df = Size_summary.mean()
Size_summary_df.head()

C:\Users\schow\AppData\Local\Temp\ipykernel_12736\3874711228.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Size_summary["Size Bins"] = pd.cut(SchoolSummary_df["Total Students"], bins, labels=bin_names)


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Size Bins,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [217]:
#Scores by school type
Type_summary = SchoolSummary_df[["School Names","School Type","Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
Type_summary = Type_summary.groupby("School Type")
Type_summary.mean()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
